# FourD.cpp
[Joshua Marshall Moore](joshua.moore@leudla.net)
March 31st, 2019


The goal here is to arrive at a dynamic, force directed layout written in cpp, that I can port to WebAssembly. The code is written in C++17, on a CERN's Cling C++ kernel.

This code will not be pretty. Please don't evaluate it for prettiness, I wrote it over a weekend. I'm sure there could be more consts, and more private members, but I just wanted to get this going.

I will need the following classes: 

1. Vertex
2. Edge
3. Graph
4. BarnesHutNode3

This is the order in which you should read this document. Each class is followed by a few (informal) tests, as appropriate. 




That should be enough for now. Let's begin: 

In [1]:
#include "gmtl/gmtl/Vec.h"
#include "gmtl/gmtl/VecOps.h"
#include "gmtl/gmtl.h"
#include <vector>
#include <random>
#include <iostream>
#include <chrono> 
#include <map>
#include <sstream>

using namespace std;
using namespace std::chrono; 
using namespace gmtl;

## Constants

In [2]:
float _repulsion = 25.0;
float _epsilon = 0.1;
float _inner_distance = 0.36;
float _attraction = 0.05;
float _friction = 0.60;
float _gravity = 10;
 
float _min_start_pos = -1.0f;
float _max_start_pos = 1.0f;

class Settings {
  public:
  
    Settings(float _repulsion, float _epsilon, float _inner_distance, float _attraction, float _friction, float _gravity, float _min_start_pos, float _max_start_pos){
      repulsion = _repulsion;
      epsilon = _epsilon;
      inner_distance = _inner_distance;
      attraction = _attraction;
      friction = _friction;
      gravity = _gravity;
      
      min_start_pos = _min_start_pos;
      max_start_pos = _max_start_pos;
    }
  
    float repulsion;
    float epsilon;
    float inner_distance;
    float attraction;
    float friction;
    float gravity;
    float min_start_pos;
    float max_start_pos;
};

Settings SETTINGS(_repulsion, _epsilon, _inner_distance, _attraction, _friction, _gravity, _min_start_pos, _max_start_pos);

## The Randomator class
The Randomator will return a floating point number in the range -1.0f to 1.0f for use in the Vertex constructor.

In [3]:

template <typename T>
class Randomator {
  public:
    static inline random_device r;
    static inline default_random_engine * engine = new default_random_engine(Randomator::r());
    uniform_real_distribution<T> dist;
  
    Randomator(T min, T max){
      this->dist = uniform_real_distribution<T>(min, max);
    }

    T get(){
      return this->dist(*Randomator::engine);
    }
};

In [4]:
Randomator<float> ra( -1, 1);
cout << ra.get() << " " << ra.get() << endl;

0.915759 -0.840898


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fe0090ee640


In [5]:
float length(gmtl::Vec3f v){
  return sqrt(v[0]*v[0] + v[1]*v[1] + v[2]*v[2]);
};


In [6]:
gmtl::Vec3f vec1(ra.get(), ra.get(), ra.get());
gmtl::Vec3f vec2(ra.get(), ra.get(), ra.get());

cout << vec1 << " " << vec2 << endl;
cout << vec1 - vec2 << endl;

(-0.97689, -0.596359, 0.999493) (0.485975, -0.224361, -0.828224)
(-1.46287, -0.371998, 1.82772)


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fe0090ee640


## The Vertex class

In [7]:
template <class T>
class Vertex {
  public:
    static inline Randomator<float> r = Randomator<float>(-1.0f, 1.0f);
    static inline int _id = 0;
  
    Vertex(T userdata){
      data = userdata;
      position = gmtl::Vec3f(Vertex::r.get(), Vertex::r.get(), Vertex::r.get());
      id = Vertex::_id++;
    }

    int id;
    T data;
    gmtl::Vec3f position;
    gmtl::Vec3f velocity;
    gmtl::Vec3f acceleration;
  
    gmtl::Vec3f repulsion_forces;
    gmtl::Vec3f attraction_forces;
  
    static gmtl::Vec3f pairwise_repulsion(const gmtl::Vec3f& one, const gmtl::Vec3f& other){
      gmtl::Vec3f diff = one - other;
      // gmtl::Vec3f diff = *(this->position) - *(other->position);
      float abs_diff = length(diff);
      return  (SETTINGS.repulsion / 
               ((SETTINGS.epsilon + abs_diff)*(SETTINGS.epsilon + abs_diff)) * 
               (diff / abs_diff));
    }
  
    bool operator==(const Vertex<T>& other){
      return other.id == id;
    }
  
    string toString(){
      stringstream ss;
      ss << "Vertex " << id;
      return ss.str();
    }
};

## Testing the Pairwise Repulsion Function

In [8]:
Vertex<int> v1(1);
Vertex<int> v2(2);
cout << Vertex<int>::pairwise_repulsion(v1.position, v2.position) << std::endl;

(-2.67406, 0.612305, -5.56118)


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fe0090ee640


## The Edge Class
An edge connects a source and a target vertex.

In [9]:
template <class T>
class Edge {
  public:
    static inline int _id = 0;
  
    Edge(Vertex<T>* _source, Vertex<T>* _target){
      source = _source;
      target = _target;
      id = Edge::_id++;
    };

    int id;
    Vertex<T>* source;
    Vertex<T>* target;
    bool directed = false;
  
    string toString(){
      stringstream ss;
      ss << "Edge " << id;
      return ss.str();
    }
};

## The Barnes Hut Tree

A [Barnes Hut Tree](https://en.wikipedia.org/wiki/Barnes%E2%80%93Hut_simulation) is a three dimensional data structure. Like a binary tree for each axis, it sorts the received data into left/right up/down and in/out by comparing the received position with the calculated center of a node. If a certain distance to the center is overcome, the vertex argument is placed into its own tree node. 

In [10]:
template <class T>
class BarnesHutNode3 {
  public:
    vector<Vertex<T>> inners;
    map<string, BarnesHutNode3<T>*> outers;
    gmtl::Vec3f center_sum;
    int count = 0;
  
    gmtl::Vec3f center(){
      return this->center_sum / (float)this->count;
    }
  
    void place_inner(Vertex<T>& vertex){
      this->inners.push_back(vertex);
      this->center_sum += vertex.position;
    }
  
    void place_outer(Vertex<T>& vertex){
      string octant = this->get_octant(vertex.position);
      // this->outers.insert(make_pair(octant, new BarnesHutNode3<T>()));
      this->outers[octant] = new BarnesHutNode3<T>();
      this->outers[octant]->insert(vertex);
    }
  
    void insert(Vertex<T>& vertex){
      if(this->inners.size() == 0){
        this->place_inner(vertex);
      }else{
        gmtl::Vec3f center = this->center();
        gmtl::Vec3f pos = vertex.position;
        float distance = sqrt((center[0] - pos[0])*(center[0] - pos[0]) + 
                             (center[1] - pos[1])*(center[1] - pos[1]) +
                             (center[2] - pos[2])*(center[2] - pos[2]));
        
        if(distance <= SETTINGS.inner_distance){
          this->place_inner(vertex);
        }else{
          this->place_outer(vertex);
        }
      }
      
      this->count++;
    }
  
    string get_octant(gmtl::Vec3f& position){
      gmtl::Vec3f center = this->center();
      string x = center[0] < position[0] ? "l" : "r";
      string y = center[1] < position[1] ? "u" : "d";
      string z = center[2] < position[2] ? "i" : "o";
      return x+y+z;
    }
  
    void estimate(Vertex<T>& vertex, gmtl::Vec3f& force, gmtl::Vec3f (*force_fn)(const gmtl::Vec3f& p1, const gmtl::Vec3f& p2)){
      if(find(this->inners.begin(), this->inners.end(), vertex) != this->inners.end()){ // todo: make better, maintain a set or something
        for(auto i=0; i<this->inners.size(); i++){
          if(this->inners[i].id != vertex.id){
            gmtl::Vec3f f = force_fn(vertex.position, this->inners[i].position);
            force += f;
          }
        }
      }else{
        gmtl::Vec3f f = force_fn(vertex.position, this->center()) * (float)this->inners.size();
        force += f;
      }
      
      for(auto &it : this->outers){
        this->outers[it.first]->estimate(vertex, force, force_fn);
      }
    }
  
    string toString(){
      return "BarnesHutNode3";
    }
  
    unsigned int size(){
      return this->count;
    }
};

## The Graph Class

In [11]:
template <class T>
class Graph {
  public:
    void add_vertex(Vertex<T> vertex){
      V.push_back(vertex);
    }
  
    void add_edge(Edge<T> edge){
      E.push_back(edge);
    }
  
    void remove_vertex(Vertex<T> vertex){
      V.erase(vertex);
    }
  
    void remove_edge(Edge<T> edge){
      E.erase(edge);
    }
  
    vector<Vertex<T>> V;
    vector<Edge<T>> E;
  
    void layout(){
      // calculate repulsions
      
      BarnesHutNode3<T> tree = BarnesHutNode3<T>();
      for(Vertex<T>& vertex : this->V){
        tree.insert(vertex);
      }
      for(Vertex<T>& vertex : this->V){
        vertex.repulsion_forces = gmtl::Vec3f();
        tree.estimate(
          vertex,
          vertex.repulsion_forces,
          &Vertex<T>::pairwise_repulsion);
      }
      
      // calculate attractions 
      for(Edge<T> edge : this->E){
        gmtl::Vec3f attraction = (edge.source->position - edge.target->position) * (-1 * SETTINGS.attraction);
        if(edge.directed){
          gmtl::Vec3f sp = edge.source->position;
          gmtl::Vec3f tp = edge.target->position;
          
          float distance = sqrt((sp[0] - tp[0])*(sp[0] - tp[0]) + 
                                      (sp[1] - tp[1])*(sp[1] - tp[1]) +
                                      (sp[2] - tp[2])*(sp[2] - tp[2]));
          gmtl::Vec3f gravity = gmtl::Vec3f(0.0f, SETTINGS.gravity/distance, 0.0f);
          edge.source->attraction_forces -= attraction;
          edge.target->attraction_forces += attraction;
        }
      }
      
      // update vertices
      for(Vertex<T>& vertex : this->V){
        gmtl::Vec3f friction = vertex.velocity * SETTINGS.friction;
        vertex.acceleration += vertex.repulsion_forces - vertex.attraction_forces - friction;
        vertex.velocity += vertex.acceleration;
        vertex.position += vertex.velocity;
      }
    }
};

## A Quick Test Of The Graph

In [12]:
Graph<int> graph;

The graph instantiates.

In [13]:
cout << "Empty graph layout test... ";
graph.layout();
cout << "done." << endl;

Empty graph layout test... done.


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fe0090ee640


The layout function runs when called on an empty graph.

In [14]:
#include <iostream>
#include <chrono>
using namespace std;
using namespace std::chrono;

int NUM_VERTICES = 10;
int NUM_EDGES = 20;

auto start = high_resolution_clock::now(); 
for(int i=0; i<NUM_VERTICES; i++){
  graph.add_vertex(Vertex<int>(i));
}

for(int n=0; n<NUM_EDGES; ++n){
  graph.add_edge(Edge<int>(&graph.V[random() % graph.V.size()], &graph.V[random() % graph.V.size()]));
}
auto stop = high_resolution_clock::now(); 

cout << "Added " << NUM_VERTICES << " vertices in " << duration<double>(stop - start).count() << " seconds." << endl; 

Added 10 vertices in 2.3905e-05 seconds.


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fe0090ee640


In [15]:
cout << graph.V.size() << " vertices" << endl;
cout << graph.E.size() << " edges" << endl;

10 vertices
20 edges


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fe0090ee640


In [16]:
#include <iostream>
#include <chrono>
using namespace std;
using namespace std::chrono;

auto start2 = high_resolution_clock::now();
gmtl::Vec3f all;

for(auto i=0; i<graph.V.size(); i++){
  for(auto j=0; j<graph.V.size(); j++){
    if(i<j){
      all += Vertex<int>::pairwise_repulsion(graph.V[i].position, graph.V[j].position);
    }
  }
}
auto stop2 = high_resolution_clock::now();
cout << "Took " << duration<double>(stop2 - start2).count() << "s : " << " calculate "<< (all / (float)graph.V.size()) << endl;

Took 1.6496e-05s :  calculate (-7.63782, 33.2137, 31.0779)


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fe0090ee640


## Testing the insert and estimate functions

In [17]:
BarnesHutNode3<int> tree;

The tree instantiates.

In [18]:
using namespace std;
using namespace std::chrono;

auto start3 = high_resolution_clock::now();
for(auto i=0; i<graph.V.size(); i++){
  tree.insert(graph.V[i]);
}
auto stop3 = high_resolution_clock::now();

cout << "Took " << duration<double>(stop3 - start3).count() << "s to insert " << graph.V.size() << " vertices." << endl;

Took 3.8266e-05s to insert 10 vertices.


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fe0090ee640


Tree insertion successful.

In [19]:
using namespace std;
using namespace std::chrono;

for(auto vertex : graph.V){
  auto start4 = high_resolution_clock::now();
  gmtl::Vec3f force;
  tree.estimate(vertex, force, &Vertex<int>::pairwise_repulsion);
  auto stop4 = high_resolution_clock::now();
  cout << "Took " << duration<double>(stop4 - start4).count() << "s to estimate " << force << endl;
}

Took 1.8568e-05s to estimate (53.6967, 92.4281, -16.0417)
Took 1.0687e-05s to estimate (-49.3802, 19.4952, 267.747)
Took 7.092e-06s to estimate (-79.2401, -26.7276, 6.07264)
Took 7.299e-06s to estimate (49.9522, 15.1778, -31.7831)
Took 6.318e-06s to estimate (-64.9581, 103.621, -5.45273)
Took 7.14e-06s to estimate (78.3768, 30.0358, -61.4725)
Took 6.289e-06s to estimate (66.2188, 108.825, 153.093)
Took 6.723e-06s to estimate (-13.5154, -56.7096, -19.1831)
Took 6.268e-06s to estimate (33.9325, -131.365, -11.3625)
Took 6.66e-06s to estimate (24.0491, -16.576, -48.976)


In [20]:
long rs[] = {random(), random()};
rs

(long [2]) { 149798315, 2038664370 }


This proves that the random number generator indeed returns two different numbers for two different function calls.

## Testing graph.layout()

In [21]:
#include <iostream>
#include <chrono>
#include <iostream>
using namespace std;
using namespace std::chrono;

NUM_VERTICES = 10000;
NUM_EDGES = NUM_VERTICES * 3;

time_point start5 = high_resolution_clock::now();
Graph<int> g = Graph<int>();
for(int i=0; i<NUM_VERTICES; i++){
  g.add_vertex(Vertex<int>(i));
}
for(int i=0; i<NUM_EDGES; i++){
  g.add_edge(Edge<int>(&g.V[random() % g.V.size()], &g.V[random() % g.V.size()]));
}
g.layout();
time_point stop5 = high_resolution_clock::now();

cout << "g.layout() took " << duration<double>(stop5 - start5).count() << "s with " << g.V.size() << " vertices and " << g.E.size() << " edges." << endl;

g.layout() took 0.242327s with 10000 vertices and 30000 edges.


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fe0090ee640


## Average Vertex Position

The average vertex position is a tool I can use for analysis of the values over multiple layout() calls. 


In [22]:
template <class T>
gmtl::Vec3f avg_position(const Graph<T>& graph){
  gmtl::Vec3f position;
  for(auto vertex : graph.V){
    position += vertex.position;
  }
  return position / (float)graph.V.size();
}

/* only call this function with type int for now */
template <class T>
vector<gmtl::Vec3f> average_positions(int iterations, int vertices, int edges){
  vector<gmtl::Vec3f> history;
  Graph<T> h;
  for(int i=0; i<vertices; i++){
    h.add_vertex(Vertex<T>(i));
  }
  for(int i=0; i<edges; i++){
    h.add_edge(Edge<T>(&h.V[random() % h.V.size()], &h.V[random() % h.V.size()]));
  }

  for(int i=0; i<iterations; i++){
    h.layout();
    cout << ".";
    history.push_back(avg_position(h));
  }
  
  return history;
};


In [23]:
gmtl::Vec3f pos = avg_position<int>(g);
cout << pos << endl;

(62.5076, 25.4216, 103.589)


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fe0090ee640


In [24]:
/*
cout << avg_position<int>(g) << endl;
for(auto n=0; n<100; n++){
  g.layout();
  cout << ".";
}
cout << endl;
cout << avg_position<int>(g) << endl;
*/

This establishes that the graph's layout function has an effect on its vertices' average positions.

In [25]:
/*
auto history = average_positions<int>(200, 2, 1);
for(auto i=0; i<history.size(); i++){
  cout << length(history[i]) << endl;
}
*/

The graph is oscillating. 

Let's fix that!

In [26]:
#include <vector>
template <class T>
class Experiment {
  public:
    Experiment<T>(T& _variable, const std::vector<T>& _values){
      variable = &_variable;
      values = _values;
      
      for(auto value : values){
        *(this->variable) = value;
        this->histories.push_back(average_positions<int>(50, 10, 30));
      }
    }
  
    T* variable;
    std::vector<T> values;
    std::vector<std::vector<gmtl::Vec3f>> histories;
};

In [27]:
#include <iostream>
#include <chrono>
#include <iostream>
#include <vector>
using namespace std;
using namespace std::chrono;

cout << "attraction" << endl;
const std::vector<float> values = vector<float>({50.0f, 5.0f, 0.5f, 0.05f, 0.005f, 0.0005f});
Experiment<float> e = Experiment<float>(SETTINGS.friction, values);
for(auto i=0; i<e.values.size(); i++){
  
  for(auto j=0; j<e.histories[i].size(); j++){
    cout << i << " " << e.values[i] << " " << j << " " << length(e.histories[i][j]) << endl;
  }
}
cout << endl;

attraction
............................................................................................................................................................................................................................................................................................................0 50 0 21.2567
0 50 1 991.471
0 50 2 47598.3
0 50 3 2.2837e+06
0 50 4 1.0957e+08
0 50 5 5.25707e+09
0 50 6 2.5223e+11
0 50 7 1.21018e+13
0 50 8 5.80633e+14
0 50 9 2.78583e+16
0 50 10 1.33662e+18
0 50 11 inf
0 50 12 inf
0 50 13 inf
0 50 14 inf
0 50 15 inf
0 50 16 inf
0 50 17 inf
0 50 18 inf
0 50 19 inf
0 50 20 inf
0 50 21 inf
0 50 22 -nan
0 50 23 -nan
0 50 24 -nan
0 50 25 -nan
0 50 26 -nan
0 50 27 -nan
0 50 28 -nan
0 50 29 -nan
0 50 30 -nan
0 50 31 -nan
0 50 32 -nan
0 50 33 -nan
0 50 34 -nan
0 50 35 -nan
0 50 36 -nan
0 50 37 -nan
0 50 38 -nan
0 50 39 -nan
0 50 40 -nan
0 50 41 -nan
0 50 42 -nan
0 50 43 -nan
0 50 44 -nan
0 50 45 -nan
0 50 46 -nan
0 50 47 -nan
0 50 48 -nan
0 50 49 -na

(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fe0090ee640
